# DS 7331 - Machine Learning - Course Project
Team: Allison Roderick, Jayson Barker, Sandesh Ojha, Brandon Croom

Lab Number: Mini Project

## Model Creation/Parameters - 50pts

In [ ]:
##TO DO:
# - Add Zip, City, State fill in to EDA
# - Add Time Duration cleanup to EDA
# - Determine issue with Weather Data fill in
# - Add SVM item to Mini Project

In [1]:
# Import libraries for data analysis
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt

In [ ]:
#Run the initial EDA from the first lab for the project. 
#NOTE: We've commented out the graphs and analysis portions from the EDA just to make things run faster, hence the "Skinny"
#version of the filename
%run ./JB_AR_BC_SO_Project1_Skinny.ipynb

Initial Shape:  (2974335, 49)
Deduplication Shape:  (2974335, 49)
No Duplicate Data Found
['MapQuest' 'MapQuest-Bing' 'Bing']
['Calm' 'SW' 'SSW' 'WSW' 'WNW' 'NW' 'West' 'NNW' 'NNE' 'South' 'North'
 'Variable' 'SE' 'SSE' 'ESE' 'East' 'NE' 'ENE' 'E' 'W' nan 'S' 'VAR'
 'CALM' 'N']
['Calm' 'SW' 'SSW' 'WSW' 'WNW' 'NW' 'W' 'NNW' 'NNE' 'S' 'N' 'VAR' 'SE'
 'SSE' 'ESE' 'E' 'NE' 'ENE' nan 'CALM']
(2974335, 41)
ID                        object
Severity                   int64
Start_Time                object
End_Time                  object
Start_Lat                float64
Start_Lng                float64
Distance(mi)             float64
Side                      object
City                      object
County                    object
State                     object
Zipcode                   object
Country                   object
Timezone                  object
Airport_Code              object
Temperature(F)           float64
Wind_Chill(F)            float64
Humidity(%)              float64
Pr

C:\Users\croomb\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
#Define helper functions
def getMedianTemp(row):
    month = row['Start_Month']
    state = row['State']
    
    result = medianTempByStateMonth[state,month]
    
    if not result:
        result = medianTempByMonth[month]
        
    return result

def getMedianPrecip(row):
    month = row['Start_Month']
    state = row['State']
    
    result = medianPrecipByStateMonth[state,month]
    
    if not result:
        result = medianPrecipByMonth[month]
    
    return result

def getMedianPressure(row):
    month = row['Start_Month']
    state = row['State']
    
    result = medianPressureByStateMonth[state,month]
    
    if not result:
        result = medianPrecipByMonth[month]
    
    return result

def getMedianWindChill(row):
    month = row['Start_Month']
    state = row['State']
    
    result = medianWindChillByStateMonth[state,month]
    
    if not result:
        medianWindChillByMonth[month]
    
    return result

def getMedianVis(row):
    month = row['Start_Month']
    state = row['State']
    
    result = medianVisByStateMonth[state,month]
    
    if not result:
        result = meduanVisByMonth[month]
    
    return result

def getMedianWindSpeed(row):
    month = row['Start_Month']
    state = row['State']
    
    result = medianWindSpeedByStateMonth[state,month]
    
    if not result:
        result = medianWindSpeedByMonth[month]
    
    return result

def getMedianHumidity(row):
    month = row['Start_Month']
    state = row['State']
    
    result = medianHumidityByStateMonth[state,month]
    
    if not result:
        result = medianHumidityByMonth[month]
    
    return result

In [4]:
# Copy the dataframe from the original as we'll be manipulating it for analysis
df2 = df.copy()

In [5]:
#Build the test/train split. Use a split of 80/20
#Get the values for Y
y = df2['Severity'].values

#Delete off the values for Y so they don't get duplicated in the split
del df2['Severity']

#get everything else that we want to predict off of
x = df2.values

X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=0)

In [6]:
#Convert the train/test splits back to dataframes so we can impute missing data. 
df_X_train = pd.DataFrame.from_records(X_train)
df_X_test = pd.DataFrame.from_records(X_test)
df_X_train.columns = df2.columns
df_X_test.columns = df2.columns

In [10]:
df_X_train.head()

,ID,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,PointOfInterest,Bump,Crossing,Yield,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Start_Year,Start_Month,Start_Day,Start_Hour,Start_Weekday,Start_Date,End_Year,End_Month,End_Day,End_Hour,End_Weekday,End_Date,Time_Duration(min),Season
0,A-1972838,2017-12-19 16:26:19,2017-12-19 16:56:05,32.839256,-97.264183,0.000,R,Haltom City,Tarrant,TX,76117,US,US/Central,KFWD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day,2017,Dec,19,16,Tue,2017-12-19,2017,Dec,19,16,Tue,2017-12-19,30.0,Fall
1,A-327130,2017-02-03 17:42:15,2017-02-03 18:12:15,41.513672,-72.770172,0.010,R,Meriden,New Haven,CT,06450,US,US/Eastern,KMMK,32.0,26.4,38.0,30.09,10.0,WNW,5.8,NaN,Partly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Day,Day,2017,Feb,3,17,Fri,2017-02-03,2017,Feb,3,18,Fri,2017-02-03,30.0,Winter
2,A-2180962,2017-07-27 23:13:11,2017-07-27 23:42:17,34.186272,-118.399971,0.640,R,North Hollywood,Los Angeles,CA,91606,US,US/Pacific,KBUR,70.0,NaN,73.0,29.88,10.0,ESE,10.4,NaN,Clear,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2017,Jul,27,23,Thu,2017-07-27,2017,Jul,27,23,Thu,2017-07-27,29.0,Summer
3,A-2071708,2017-10-11 18:37:27,2017-10-11 19:04:59,39.964279,-82.983139,0.480,R,Columbus,Franklin,OH,43215,US,US/Eastern,KCMH,66.0,NaN,78.0,30.02,10.0,WNW,13.8,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day,2017,Oct,11,18,Wed,2017-10-11,2017,Oct,11,19,Wed,2017-10-11,28.0,Fall
4,A-2412608,2017-03-13 16:37:53,2017-03-13 22:37:53,36.035200,-79.794000,0.287,R,Greensboro,Guilford,NC,27406,US,US/Eastern,KGSO,37.9,30.9,86.0,30.13,2.0,ENE,10.4,0.0,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day,2017,Mar,13,16,Mon,2017-03-13,2017,Mar,13,22,Mon,2017-03-13,360.0,Winter


In [7]:
#Check the counts of our training dataset to see how many of the values are missing
print("Temp: ",df_X_train['Temperature(F)'].isnull().sum())
print("Precip: ",df_X_train['Precipitation(in)'].isnull().sum())
print("Pressure: ",df_X_train['Pressure(in)'].isnull().sum())
print("Wind Chill: ",df_X_train['Wind_Chill(F)'].isnull().sum())
print("Viz: ", df_X_train['Visibility(mi)'].isnull().sum())
print("Wind Speed: ", df_X_train['Wind_Speed(mph)'].isnull().sum())
print("Humidity: ", df_X_train['Humidity(%)'].isnull().sum())

Temp:  44881
Precip:  1599144
Pressure:  38584
Wind Chill:  1482737
Viz:  52570
Wind Speed:  352684
Humidity:  47405


In [8]:
#Check the counts of our test dataset to see how many of the values are missing
print("Temp: ",df_X_test['Temperature(F)'].isnull().sum())
print("Precip: ",df_X_test['Precipitation(in)'].isnull().sum())
print("Pressure: ",df_X_test['Pressure(in)'].isnull().sum())
print("Wind Chill: ",df_X_test['Wind_Chill(F)'].isnull().sum())
print("Viz: ", df_X_test['Visibility(mi)'].isnull().sum())
print("Wind Speed: ", df_X_test['Wind_Speed(mph)'].isnull().sum())
print("Humidity: ", df_X_test['Humidity(%)'].isnull().sum())

Temp:  11182
Precip:  399214
Pressure:  9558
Wind Chill:  369886
Viz:  13121
Wind Speed:  88156
Humidity:  11768


In [9]:
#Given that we're looking at weather based conditions take median values for each of the features. Initially start by
#determining median by state and start month. To also cover any missing values that may not be present with this 
#combination calculate the overall median by month. 

medianTempByStateMonth = df_X_train.groupby(['State','Start_Month'])['Temperature(F)'].median(skipna=True)
medianPrecipByStateMonth  = df_X_train.groupby(['State','Start_Month'])['Precipitation(in)'].median(skipna=True)
medianPressureByStateMonth  = df_X_train.groupby(['State','Start_Month'])['Pressure(in)'].median(skipna=True)
medianWindChillByStateMonth  = df_X_train.groupby(['State','Start_Month'])['Wind_Chill(F)'].median(skipna=True)
medianVisByStateMonth  = df_X_train.groupby(['State','Start_Month'])['Visibility(mi)'].median(skipna=True)
medianWindSpeedByStateMonth  = df_X_train.groupby(['State','Start_Month'])['Wind_Speed(mph)'].median(skipna=True)
medianHumidityByStateMonth  = df_X_train.groupby(['State','Start_Month'])['Humidity(%)'].median(skipna=True)

medianTempByMonth = df_X_train.groupby(['Start_Month'])['Temperature(F)'].median(skipna=True)
medianPrecipByMonth  = df_X_train.groupby(['Start_Month'])['Precipitation(in)'].median(skipna=True)
medianPressureByMonth  = df_X_train.groupby(['Start_Month'])['Pressure(in)'].median(skipna=True)
medianWindChillByMonth  = df_X_train.groupby(['Start_Month'])['Wind_Chill(F)'].median(skipna=True)
medianVisByMonth  = df_X_train.groupby(['Start_Month'])['Visibility(mi)'].median(skipna=True)
medianWindSpeedByMonth  = df_X_train.groupby(['Start_Month'])['Wind_Speed(mph)'].median(skipna=True)
medianHumidityByMonth  = df_X_train.groupby(['Start_Month'])['Humidity(%)'].median(skipna=True)

In [10]:
#Apply the median values to the training and test data frames

df_X_train.loc[pd.isnull(df_X_train['Temperature(F)']),'Temperature(F)'] = df_X_train.swifter.apply(getMedianTemp, axis=1)
df_X_train.loc[pd.isnull(df_X_train['Precipitation(in)']),'Precipitation(in)'] = df_X_train.swifter.apply(getMedianPrecip,axis=1)
df_X_train.loc[pd.isnull(df_X_train['Pressure(in)']),'Pressure(in)'] = df_X_train.swifter.apply(getMedianPressure,axis=1)
df_X_train.loc[pd.isnull(df_X_train['Wind_Chill(F)']),'Wind_Chill(F)'] = df_X_train.swifter.apply(getMedianWindChill,axis=1)
df_X_train.loc[pd.isnull(df_X_train['Visibility(mi)']),'Visibility(mi)'] = df_X_train.swifter.apply(getMedianVis,axis=1)
df_X_train.loc[pd.isnull(df_X_train['Wind_Speed(mph)']),'Wind_Speed(mph)'] = df_X_train.swifter.apply(getMedianWindSpeed,axis=1)
df_X_train.loc[pd.isnull(df_X_train['Humidity(%)']),'Humidity(%)'] = df_X_train.swifter.apply(getMedianHumidity,axis=1)

In [11]:
df_X_test.loc[pd.isnull(df_X_test['Temperature(F)']),'Temperature(F)'] = df_X_test.swifter.apply(getMedianTemp,axis=1)
df_X_test.loc[pd.isnull(df_X_test['Precipitation(in)']),'Precipitation(in)'] = df_X_test.swifter.apply(getMedianPrecip,axis=1)
df_X_test.loc[pd.isnull(df_X_test['Pressure(in)']),'Pressure(in)'] = df_X_test.swifter.apply(getMedianPressure,axis=1)
df_X_test.loc[pd.isnull(df_X_test['Wind_Chill(F)']),'Wind_Chill(F)'] = df_X_test.swifter.apply(getMedianWindChill,axis=1)
df_X_test.loc[pd.isnull(df_X_test['Visibility(mi)']),'Visibility(mi)'] = df_X_test.swifter.apply(getMedianVis,axis=1)
df_X_test.loc[pd.isnull(df_X_test['Wind_Speed(mph)']),'Wind_Speed(mph)'] = df_X_test.swifter.apply(getMedianWindSpeed,axis=1)
df_X_test.loc[pd.isnull(df_X_test['Humidity(%)']),'Humidity(%)'] = df_X_test.swifter.apply(getMedianHumidity,axis=1)

In [12]:
#Check the counts of our training dataset to ensure we cleared out all nulls
print("Temp: ",df_X_train['Temperature(F)'].isnull().sum())
print("Precip: ",df_X_train['Precipitation(in)'].isnull().sum())
print("Pressure: ",df_X_train['Pressure(in)'].isnull().sum())
print("Wind Chill: ",df_X_train['Wind_Chill(F)'].isnull().sum())
print("Viz: ", df_X_train['Visibility(mi)'].isnull().sum())
print("Wind Speed: ", df_X_train['Wind_Speed(mph)'].isnull().sum())
print("Humidity: ", df_X_train['Humidity(%)'].isnull().sum())

Temp:  0
Precip:  14
Pressure:  0
Wind Chill:  11
Viz:  0
Wind Speed:  1
Humidity:  0


In [13]:
#Check the counts of our test dataset to ensure we cleared out all nulls
print("Temp: ",df_X_test['Temperature(F)'].isnull().sum())
print("Precip: ",df_X_test['Precipitation(in)'].isnull().sum())
print("Pressure: ",df_X_test['Pressure(in)'].isnull().sum())
print("Wind Chill: ",df_X_test['Wind_Chill(F)'].isnull().sum())
print("Viz: ", df_X_test['Visibility(mi)'].isnull().sum())
print("Wind Speed: ", df_X_test['Wind_Speed(mph)'].isnull().sum())
print("Humidity: ", df_X_test['Humidity(%)'].isnull().sum())

Temp:  0
Precip:  3
Pressure:  0
Wind Chill:  3
Viz:  0
Wind Speed:  0
Humidity:  0


In [57]:
print(df_X_test[df_X_test['Precipitation(in)'].isnull()])

print(medianPrecipByStateMonth['ND','Aug'])

               ID          Start_Time            End_Time  Start_Lat  \
299572  A-2183081 2017-08-01 08:02:16 2017-08-01 08:31:50  46.899204   
415815  A-2274693 2016-09-06 00:26:23 2016-09-06 06:26:23  46.828960   
430902  A-1401002 2018-09-06 08:08:20 2018-09-06 08:38:03  46.880718   

         Start_Lng  Distance(mi) Side       City    County State Zipcode  \
299572  -98.708099         0.000    R  Jamestown  Stutsman    ND   58401   
415815 -100.871150         0.462    R     Mandan    Morton    ND   58554   
430902  -99.637260         0.000    R     Tappen    Kidder    ND   58487   

       Country    Timezone Airport_Code  Temperature(F)  Wind_Chill(F)  \
299572      US  US/Central         KJMS            64.0            NaN   
415815      US  US/Central         KY19            55.4            NaN   
430902      US  US/Central         KJMS            53.1            NaN   

        Humidity(%)  Pressure(in)  Visibility(mi) Wind_Direction  \
299572         84.0         30.12        

In [62]:
medianPrecipByMonth

Start_Month
Apr    0.00
Aug    0.00
Dec    0.00
Feb    0.01
Jan    0.01
Jul    0.00
Jun    0.00
Mar    0.01
May    0.00
Nov    0.00
Oct    0.00
Sep    0.00
Name: Precipitation(in), dtype: float64

In [14]:
featuresList = ['Start_Lat','Start_Lng','Distance(mi)','Side','Timezone','Temperature(F)','Wind_Chill(F)',
                'Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Direction','Wind_Speed(mph)','Precipitation(in)',
                'Weather_Condition','PointOfInterest','Bump','Crossing','Yield','Junction','No_Exit','Railway',
                'Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop','Sunrise_Sunset',
                'Start_Year','Start_Month','Start_Day','Start_Hour','Start_Weekday','End_Year','End_Month','End_Day',
                'End_Hour','End_Weekday','Time_Duration(min)','Season']

df_X_test = df_X_test[featuresList]
df_X_train = df_X_train[featuresList]

In [18]:
# transform categories into dummy numerics
cols_to_transform = ['Side','Weather_Condition','Sunrise_Sunset','Season','Start_Weekday','End_Weekday','Timezone']

df_X_test = pd.get_dummies(df_X_test, columns = cols_to_transform )
df_X_train = pd.get_dummies(df_X_train, columns = cols_to_transform )

In [19]:
X_train = df_X_train.to_numpy()
X_test = df_X_test.to_numpy()

In [20]:
# logistic regression
logistic_regression= LogisticRegression(max_iter = 4000)
logistic_regression.fit(X_train,y_train)
y_pred=logistic_regression.predict(X_test)

C:\Users\croomb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


MemoryError: 

In [ ]:
# Accuracy | Accuracy = (TP+TN)/Total
print('Accuracy: ',mt.accuracy_score(y_test, y_pred))

## Model Advantages - 10pts

## Interpretation - 30pts

## SVM Insight - 10pts